En este archivo crearemos un dataframe al estilo doomies de los generos que poseemos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [2]:
data = pd.read_csv('data/peliculas_ETL.csv', low_memory=False, index_col='id')
df = data.copy()
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42196 entries, 862 to 461257
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  42196 non-null  object 
 1   budget                 42196 non-null  float64
 2   genres                 42196 non-null  object 
 3   original_language      42196 non-null  object 
 4   overview               42196 non-null  object 
 5   popularity             42196 non-null  float64
 6   production_companies   42196 non-null  object 
 7   production_countries   42196 non-null  object 
 8   release_date           42196 non-null  object 
 9   revenue                42196 non-null  float64
 10  runtime                42196 non-null  float64
 11  spoken_languages       42196 non-null  object 
 12  status                 42196 non-null  object 
 13  tagline                42196 non-null  object 
 14  title                  42196 non-null  object 
 15  vote

In [3]:
df.reset_index(inplace=True)

In [4]:
df.describe()

,id,budget,popularity,revenue,runtime,vote_average,vote_count,release_year,return
count,42196.000000,4.219600e+04,42196.000000,4.219600e+04,42196.000000,42196.000000,42196.000000,42196.000000,4.219600e+04
mean,106669.904896,4.160088e+06,2.914097,1.116670e+07,93.853302,5.631261,109.391720,1991.914399,7.095609e+04
std,111704.740881,1.721499e+07,5.744661,6.377178e+07,37.578123,1.921395,489.341742,23.839319,7.745654e+06
min,2.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1874.000000,-1.000000e+02
25%,25792.000000,0.000000e+00,0.387424,0.000000e+00,85.000000,5.000000,3.000000,1978.000000,0.000000e+00
50%,58390.500000,0.000000e+00,1.136089,0.000000e+00,95.000000,6.000000,10.000000,2001.000000,0.000000e+00
75%,152939.250000,0.000000e+00,3.717885,0.000000e+00,106.000000,6.800000,35.000000,2010.000000,0.000000e+00
max,469172.000000,3.800000e+08,547.488300,2.787965e+09,1256.000000,10.000000,14075.000000,2020.000000,1.239638e+09


In [5]:
df_reducido = df[['belongs_to_collection', 'genres',
       'overview',  'production_companies',
       'production_countries',
         'tagline', 'title',  'director', 'actors_names']].copy()

In [6]:
series_genres = pd.DataFrame(df_reducido['genres'])
series_genres.replace('N/D', '[]', inplace=True)
series_genres.head()

,genres
0,"['Animation', 'Comedy', 'Family']"
1,"['Adventure', 'Fantasy', 'Family']"
2,"['Romance', 'Comedy']"
3,"['Comedy', 'Drama', 'Romance']"
4,['Comedy']


In [7]:
generos_unicos = []
for i in range(0, series_genres.shape[0]):
    fila = series_genres.iloc[i, 0]
    fila = eval(fila)
    for genero in fila:
        if genero not in generos_unicos:
            generos_unicos.append(genero)
print(generos_unicos)
    

['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History', 'Science Fiction', 'Mystery', 'War', 'Foreign', 'Music', 'Documentary', 'Western', 'TV Movie']


In [8]:
diccionario_generos = {}
for genero in generos_unicos:
    diccionario_generos[genero] = []


for i in range(0, series_genres.shape[0]):
    fila = series_genres.iloc[i, 0]
    fila = eval(fila)
    if len(fila) == 0:
        for key in diccionario_generos:
            diccionario_generos[key].append(0)
    else:
        for key in diccionario_generos:
            if key in fila:
                diccionario_generos[key].append(1)
            else:
                diccionario_generos[key].append(0)

generos_dummies = pd.DataFrame(diccionario_generos)
generos_dummies.head(3)

,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
df_index = list(df.index)
dicc_index = {'id': df_index}
df_index = pd.DataFrame(dicc_index)
generos_dummies['id'] = df_index
generos_dummies.head(3)

,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,...,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie,id
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [10]:
generos_dummies.to_csv('data/generos_dummies')